In [ ]:
import numpy as np
import pandas as pd
import gensim
import nltk
import re
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

In [ ]:

from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
train = pd.read_csv("/content/drive/MyDrive/daaaataaa_citae/tsv/train.tsv", sep= '\t',names=["id", "explicit", "text", "label"])
dev = pd.read_csv("/content/drive/MyDrive/daaaataaa_citae/tsv/dev.tsv", sep= '\t',names=["id", "explicit", "text", "label"])
test = pd.read_csv("/content/drive/MyDrive/daaaataaa_citae/tsv/test.tsv", sep= '\t', names=["id", "explicit", "text", "label"])

In [ ]:
train.drop(train.columns[[0, 1]], axis = 1, inplace = True)

In [ ]:
test.drop(test.columns[[0, 1]], axis = 1, inplace = True)

In [ ]:
test.head()

,text,label
0,"Chapel, as well as X10 [2], UPC [3] , CoArray ...",background
1,"In addition, the result of the present study s...",result
2,Several instruments that more specifically add...,background
3,Organotypic hippocampal slice cultures\nInterf...,method
4,Activated PBMC are the basis of the standard P...,background


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text
train['text_train']= train['text'].apply(lambda x:remove_punctuation(x))
train['labels_train'] = train['label'].replace(['background','method','result'],[0,1,2])
train.head()

,text,label,text_train,labels_train
0,"However, how frataxin interacts with the Fe-S ...",background,However how frataxin interacts with the FeS cl...,0
1,"In the study by Hickey et al. (2012), spikes w...",background,In the study by Hickey et al 2012 spikes were ...,0
2,"The drug also reduces catecholamine secretion,...",background,The drug also reduces catecholamine secretion ...,0
3,By clustering with lowly aggressive close kin ...,background,By clustering with lowly aggressive close kin ...,0
4,Ophthalmic symptoms are rare manifestations of...,background,Ophthalmic symptoms are rare manifestations of...,0


In [ ]:
#storing the puntuation free text
test['text_test']= test['text'].apply(lambda x:remove_punctuation(x))
test['labels_test'] = test['label'].replace(['background','method','result'],[0,1,2])
test.head()

,text,label,text_test,labels_test
0,"Chapel, as well as X10 [2], UPC [3] , CoArray ...",background,Chapel as well as X10 2 UPC 3 CoArray Fortran...,0
1,"In addition, the result of the present study s...",result,In addition the result of the present study su...,2
2,Several instruments that more specifically add...,background,Several instruments that more specifically add...,0
3,Organotypic hippocampal slice cultures\nInterf...,method,Organotypic hippocampal slice culturesnInterfa...,1
4,Activated PBMC are the basis of the standard P...,background,Activated PBMC are the basis of the standard P...,0


In [ ]:
#converting to lower case
train['text_train']= train['text_train'].apply(lambda x: x.lower())

In [ ]:
test['text_test']= test['text_test'].apply(lambda x: x.lower())

In [ ]:
test.head()

,text,label,text_test,labels_test
0,"Chapel, as well as X10 [2], UPC [3] , CoArray ...",background,chapel as well as x10 2 upc 3 coarray fortran...,0
1,"In addition, the result of the present study s...",result,in addition the result of the present study su...,2
2,Several instruments that more specifically add...,background,several instruments that more specifically add...,0
3,Organotypic hippocampal slice cultures\nInterf...,method,organotypic hippocampal slice culturesninterfa...,1
4,Activated PBMC are the basis of the standard P...,background,activated pbmc are the basis of the standard p...,0


In [ ]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def tokenize(word):
   word = nltk.word_tokenize(word)
   return word

In [ ]:
train['tokenized']= train['text_train'].apply(lambda x: tokenize(x))
test['tokenized']= test['text_test'].apply(lambda x: tokenize(x))

In [ ]:
def remove_stopwords(texts):
    output= [i for i in texts if i not in stopwords]
    return output

In [ ]:
#removing stop words
train['tokenized']= train['tokenized'].apply(lambda x:remove_stopwords(x))
test['tokenized']= test['tokenized'].apply(lambda x:remove_stopwords(x))

In [ ]:
from nltk.stem import WordNetLemmatizer
#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')
def lemmatizer(text):
  lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
  return lemm_text
train['stemmed']=train['tokenized'].apply(lambda x:lemmatizer(x))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
test['stemmed']=test['tokenized'].apply(lambda x:lemmatizer(x))

In [ ]:
test.head()

,text,label,text_test,labels_test,tokenized,stemmed
0,"Chapel, as well as X10 [2], UPC [3] , CoArray ...",background,chapel as well as x10 2 upc 3 coarray fortran...,0,"[chapel, well, x10, 2, upc, 3, coarray, fortra...","[chapel, well, x10, 2, upc, 3, coarray, fortra..."
1,"In addition, the result of the present study s...",result,in addition the result of the present study su...,2,"[addition, result, present, study, supports, p...","[addition, result, present, study, support, pr..."
2,Several instruments that more specifically add...,background,several instruments that more specifically add...,0,"[several, instruments, specifically, address, ...","[several, instrument, specifically, address, p..."
3,Organotypic hippocampal slice cultures\nInterf...,method,organotypic hippocampal slice culturesninterfa...,1,"[organotypic, hippocampal, slice, culturesnint...","[organotypic, hippocampal, slice, culturesnint..."
4,Activated PBMC are the basis of the standard P...,background,activated pbmc are the basis of the standard p...,0,"[activated, pbmc, basis, standard, pbmc, blast...","[activated, pbmc, basis, standard, pbmc, blast..."


In [ ]:
x_test = test.text.tolist()

In [ ]:
x_text=train.text
x_text[0]

'However, how frataxin interacts with the Fe-S cluster biosynthesis components remains unclear as direct one-to-one interactions with each component were reported (IscS [12,22], IscU/Isu1 [6,11,16] or ISD11/Isd11 [14,15]).'

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
x_tokenized = train.stemmed.tolist()
test_tokenized =test.stemmed.tolist()

In [ ]:
#generating pos tags using nltk
pos_tags = [nltk.pos_tag(token) for token in x_tokenized]

In [ ]:
pos_tags_test = [nltk.pos_tag(token) for token in test_tokenized]

In [ ]:
#storing train POS tags to the list  
tags = []
for i in pos_tags:
  tags.append([j[1] for j in i])


In [ ]:
#storing test POS tags to the list  
tags_test = []
for i in pos_tags_test:
  tags_test.append([j[1] for j in i])

In [ ]:
from gensim.models import Word2Vec

In [ ]:
import gensim

In [ ]:
w2v_model = Word2Vec(min_count=1,     #word2vec embedding for POS tags
                 window=2,
                 size=30,
                 sample=1e-5, 
                 alpha=0.01, 
                 min_alpha=0.0007, 
                 negative=0,
                 workers=2)

In [ ]:
w2v_model.build_vocab(tags, progress_per=1)
w2v_model.train(tags, total_examples=w2v_model.corpus_count, epochs=3, report_delay=1)

# getting the vectors for the Pos_tags from w2v_model
my_dict = dict({})
for index, key in enumerate(w2v_model.wv.vocab):
    my_dict[key] = w2v_model.wv[key]

In [ ]:
X_train =tags
y_train = np.array(train['labels_train'].tolist())

In [ ]:
X_test =tags_test
y_test = np.array(test['labels_test'].tolist())

In [ ]:
#building vectors for words in text from word2vec model
def buildWordVector(text, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in text:
        try:
            vec += w2v_model[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [ ]:
n_dim = 30

In [ ]:
#preparing training data
from sklearn.preprocessing import scale
train_vecs = np.concatenate([buildWordVector(z, n_dim) for z in X_train])
train_vecs = scale(train_vecs)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [ ]:
#preparing testing data
from sklearn.preprocessing import scale
test_vecs = np.concatenate([buildWordVector(z, n_dim) for z in X_test])
test_vecs = scale(test_vecs)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [ ]:
print(y_train.shape)
print(train_vecs.shape)


(8243,)
(8243, 30)


In [ ]:
#SGD Classifier
from sklearn.linear_model import SGDClassifier
lr = SGDClassifier(loss='log', penalty='l1')
lr.fit(train_vecs, y_train)
y_pred_sgd = lr.predict(test_vecs)
print(classification_report(y_test,y_pred_sgd))

print('accuracy_sgd:', accuracy_score(y_test,y_pred_sgd))
print('recall_sgd:' ,recall_score(y_test,y_pred_sgd, average='macro'))
print('precision_sgd:',precision_score(y_test,y_pred_sgd, average='macro'))
print('f1_score_sgd:',f1_score(y_test,y_pred_sgd, average='macro'))

              precision    recall  f1-score   support

           0       0.58      0.89      0.70       997
           1       0.55      0.31      0.39       605
           2       0.33      0.00      0.01       259

    accuracy                           0.57      1861
   macro avg       0.49      0.40      0.37      1861
weighted avg       0.54      0.57      0.50      1861

accuracy_sgd: 0.5749596990865127
recall_sgd: 0.3987687025892552
precision_sgd: 0.4886684981744811
f1_score_sgd: 0.36761263688338564


In [ ]:
from sklearn.datasets import load_digits
from sklearn.linear_model import Perceptron
clf = Perceptron(tol=1e-3, random_state=0)
clf.fit(train_vecs, y_train)
y_pred_percep = clf.predict(test_vecs)
print(classification_report(y_test,y_pred_percep))

print('accuracy_perceptron:', accuracy_score(y_test,y_pred_percep))
print('recall_perceptron:' ,recall_score(y_test,y_pred_percep, average='macro'))
print('precision_perceptron:',precision_score(y_test,y_pred_percep, average='macro'))
print('f1_score_perceptron:',f1_score(y_test,y_pred_percep, average='macro'))



              precision    recall  f1-score   support

           0       0.58      0.70      0.63       997
           1       0.43      0.38      0.41       605
           2       0.15      0.08      0.10       259

    accuracy                           0.51      1861
   macro avg       0.39      0.39      0.38      1861
weighted avg       0.47      0.51      0.49      1861

accuracy_perceptron: 0.5088662009672219
recall_perceptron: 0.385927475140569
precision_perceptron: 0.3887551506734381
f1_score_perceptron: 0.38123159590707667


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
cl = KNeighborsClassifier(n_neighbors=5)
cl.fit(train_vecs, y_train)
y_pred_kn = cl.predict(test_vecs)
print(classification_report(y_test,y_pred_kn))


print('accuracy_knn:', accuracy_score(y_test,y_pred_kn))
print('recall_knn:' ,recall_score(y_test,y_pred_kn, average='macro'))
print('precision_knn:',precision_score(y_test,y_pred_kn, average='macro'))
print('f1_score_knn:',f1_score(y_test,y_pred_kn, average='macro'))


              precision    recall  f1-score   support

           0       0.56      0.78      0.65       997
           1       0.43      0.29      0.34       605
           2       0.14      0.03      0.05       259

    accuracy                           0.52      1861
   macro avg       0.38      0.37      0.35      1861
weighted avg       0.46      0.52      0.47      1861

accuracy_knn: 0.5174637291778613
recall_knn: 0.36728046227453537
precision_knn: 0.37607880129731575
f1_score_knn: 0.34903574442734103


In [ ]:
mod = SVC(kernel = 'linear', C=1)
mod.fit(train_vecs, y_train)
y_preds = clf.predict(test_vecs)

print('accuracy_SVM :',accuracy_score(y_test, y_preds))
print('recall_SVM:',recall_score(y_test, y_preds, average='macro'))
print('precision_SVM:',precision_score(y_test, y_preds, average='macro'))
print('f1score_SVM:',f1_score(y_test, y_preds, average='macro'))

accuracy_SVM : 0.48199892530897365
recall_SVM: 0.3607646599348979
precision_SVM: 0.35751449355636794
f1score_SVM: 0.3545590297898058
